# BibRec

### RF Features: Country, State, Age

In [33]:
from bibrec.server.Utils import get_books, get_users, get_ratings

books = get_books("data/BX-Books.csv")
users = get_users("data/BX-Users.csv")
ratings = get_ratings("data/BX-Book-Ratings.csv")

/run/media/fab/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(path, sep=";", encoding="latin-1")


With NaN values 34.862889904962536
used mean values 34.90360581785413


/run/media/fab/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:38: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [35]:
explicit_ratings = ratings[ratings.book_rating != 0]
explicit_ratings

,user_id,isbn,book_rating
1,276726,9780155061224,5
3,276729,9780521656153,3
4,276729,9780521795029,6
6,276736,9783257224283,8
7,276737,9780600570967,6
...,...,...,...
1149771,276704,9780743211383,7
1149773,276704,9780806917696,5
1149775,276704,9781563526299,9
1149777,276709,9780515107661,10


In [36]:
df_ratings = explicit_ratings.groupby('isbn').user_id.count().sort_values(ascending=False)
# df_ratings = explicit_ratings.groupby('isbn').user_id.count().sort_values()
df_ratings = df_ratings[:1000]
df_ratings = df_ratings.reset_index()
# user_id => nr of ratings
df_ratings = df_ratings.drop('user_id',axis=1)
df_ratings

,isbn
0,9780316666343
1,9780971880108
2,9780385504201
3,9780312195519
4,9780679781585
...,...
995,978038000148
996,9780671695071
997,9780684853529
998,9780394223803


In [37]:
df_ratings = df_ratings.merge(explicit_ratings, on='isbn', how='left')
# df_ratings = test.merge(explicit_ratings, on='isbn', how='left')
df_ratings

,isbn,user_id,book_rating
0,9780316666343,277439,7
1,9780316666343,277752,7
2,9780316666343,278633,9
3,9780316666343,638,10
4,9780316666343,709,3
...,...,...,...
68510,9780446679367,223787,7
68511,9780446679367,226267,8
68512,9780446679367,242574,8
68513,9780446679367,260099,4


In [38]:
# merge datasets
df = df_ratings.merge(users[['user_id', 'country', 'state', 'age']])
df

,isbn,user_id,book_rating,country,state,age
0,9780316666343,277439,7,usa,oregon,65.000000
1,9780671027360,277439,7,usa,oregon,65.000000
2,9780316666343,277752,7,usa,louisiana,18.000000
3,9780679734772,277752,8,usa,louisiana,18.000000
4,9780316666343,278633,9,usa,utah,38.965423
...,...,...,...,...,...,...
68510,9780394223803,271077,10,canada,british columbia,49.000000
68511,9780446679367,15783,6,usa,montana,16.032971
68512,9780446679367,129135,8,NaN,oregon,52.473237
68513,9780446679367,204414,9,usa,illinois,42.000000


In [39]:
country = pd.get_dummies(df['country'],drop_first=True)
country

,"\n/a\"", greece""",albania,algeria,antarctica,argentina,aroostook,aruba,australia,austria,bahamas,...,ysa,yugoslavia,zambia,zimbabwe,", denmark",", england, united kingdom",", malaysia",", mexico",", new zealand",", portugal"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68510,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68511,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68512,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68513,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
state = pd.get_dummies(df['state'],drop_first=True)
state

,&#322;ódzkie,(alacant),*,-,.,...,"\n/a\""",\n/a`,_,a coruña,...,zuid-holland,zuidholland,zurich,zürich,álava,maryland,new york,or,tn,town of bali
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68510,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68511,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68512,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68513,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
df = pd.concat([df, country, state], axis = 1)
df

,isbn,user_id,book_rating,country,state,age,"\n/a\"", greece""",albania,algeria,antarctica,...,zuid-holland,zuidholland,zurich,zürich,álava,maryland,new york,or,tn,town of bali
0,9780316666343,277439,7,usa,oregon,65.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9780671027360,277439,7,usa,oregon,65.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9780316666343,277752,7,usa,louisiana,18.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9780679734772,277752,8,usa,louisiana,18.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9780316666343,278633,9,usa,utah,38.965423,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68510,9780394223803,271077,10,canada,british columbia,49.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68511,9780446679367,15783,6,usa,montana,16.032971,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68512,9780446679367,129135,8,NaN,oregon,52.473237,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68513,9780446679367,204414,9,usa,illinois,42.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
df = df.loc[:,~df.columns.duplicated()]
df

,isbn,user_id,book_rating,country,state,age,"\n/a\"", greece""",albania,algeria,antarctica,...,zuid-holland,zuidholland,zurich,zürich,álava,maryland,new york,or,tn,town of bali
0,9780316666343,277439,7,usa,oregon,65.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9780671027360,277439,7,usa,oregon,65.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9780316666343,277752,7,usa,louisiana,18.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9780679734772,277752,8,usa,louisiana,18.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9780316666343,278633,9,usa,utah,38.965423,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68510,9780394223803,271077,10,canada,british columbia,49.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68511,9780446679367,15783,6,usa,montana,16.032971,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68512,9780446679367,129135,8,NaN,oregon,52.473237,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68513,9780446679367,204414,9,usa,illinois,42.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
df = df.drop(['isbn', 'user_id', 'country', 'state'], axis = 1)
df

,book_rating,age,"\n/a\"", greece""",albania,algeria,antarctica,argentina,aroostook,aruba,australia,...,zuid-holland,zuidholland,zurich,zürich,álava,maryland,new york,or,tn,town of bali
0,7,65.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,65.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,18.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,18.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,38.965423,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68510,10,49.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68511,6,16.032971,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68512,8,52.473237,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68513,9,42.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
X = df.drop('book_rating', axis = 1) #Features
X

,age,"\n/a\"", greece""",albania,algeria,antarctica,argentina,aroostook,aruba,australia,austria,...,zuid-holland,zuidholland,zurich,zürich,álava,maryland,new york,or,tn,town of bali
0,65.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,65.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,38.965423,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68510,49.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68511,16.032971,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68512,52.473237,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68513,42.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
Y = df['book_rating']   # Target Variables
Y

0         7
1         7
2         7
3         8
4         9
         ..
68510    10
68511     6
68512     8
68513     9
68514     8
Name: book_rating, Length: 68515, dtype: int64

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

In [47]:
rfc = RandomForestClassifier(n_estimators=100, min_weight_fraction_leaf= 0)
rfc.fit(X_train, y_train)

RandomForestClassifier(min_weight_fraction_leaf=0)

In [48]:
rfc_pred = rfc.predict(X_test)
print(classification_report(y_test,rfc_pred))
print(accuracy_score(y_test,rfc_pred)*100)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        91
           2       0.00      0.00      0.00       118
           3       0.02      0.01      0.01       267
           4       0.02      0.01      0.01       384
           5       0.26      0.16      0.20      1767
           6       0.09      0.04      0.06      1523
           7       0.20      0.18      0.19      3345
           8       0.29      0.41      0.34      5230
           9       0.24      0.22      0.23      3829
          10       0.32      0.38      0.35      4001

    accuracy                           0.27     20555
   macro avg       0.14      0.14      0.14     20555
weighted avg       0.24      0.27      0.25     20555

26.52882510338117
